In [3]:
! pip install yfinance

In [27]:
import yfinance as yf
import sqlite3
import pandas as pd
import datetime

In [55]:


# 設定下載的股票代號與日期範圍
symbol = '2330.TW'
start = '2023-01-01'
end = '2024-01-01'

# 下載資料
data = yf.download(symbol, start=start, end=end)
print(data.head())
print(type(data))

[*********************100%***********************]  1 of 1 completed

Price                       Adj Close   Close    High     Low    Open  \
Ticker                        2330.TW 2330.TW 2330.TW 2330.TW 2330.TW   
Date                                                                    
2023-01-03 00:00:00+00:00  437.941620   453.0   453.5   443.0   446.0   
2023-01-04 00:00:00+00:00  434.557953   449.5   455.0   448.5   449.5   
2023-01-05 00:00:00+00:00  443.258789   458.5   459.5   455.0   459.0   
2023-01-06 00:00:00+00:00  443.258789   458.5   459.5   455.0   455.0   
2023-01-09 00:00:00+00:00  465.010834   481.0   481.0   467.5   468.0   

Price                        Volume  
Ticker                      2330.TW  
Date                                 
2023-01-03 00:00:00+00:00  14885824  
2023-01-04 00:00:00+00:00  19188422  
2023-01-05 00:00:00+00:00  23549581  
2023-01-06 00:00:00+00:00  20886011  
2023-01-09 00:00:00+00:00  46666263  
<class 'pandas.core.frame.DataFrame'>


In [54]:
condition = data.isnull().any(axis='columns')
data[condition]

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,2330.TW,2330.TW,2330.TW,2330.TW,2330.TW,2330.TW
Date,,,,,,


In [21]:
inserSQL = '''
insert into record (Date, "Open", High, Low, "Adj Close", 'Volume', "Close")
        VALUES ('2023-01-03 00:00:00+00:00', 446.0, 453.0, 443.0, 437.941620, 14885824,453.0)

'''
conn = sqlite3.connect('record.db')
cursor = conn.cursor()
cursor.execute(inserSQL)
conn.commit()
cursor.close()
conn.close()

In [35]:
conn = sqlite3.connect('record.db')
with conn:
        cursor = conn.cursor()
        for items in data:
            date = items['Date']
            open = float(items['Open']) if items['Open'] != '' else 0
            high = float(items['High']) if items['High'] != '' else 0
            low = float(items['Low']) if items['Low'] != '' else 0
            adj_close = float(items['Adj Close']) if items['Adj Close'] != '' else 0.0
            volume = int(items['Volume']) if items['Volume'] != '' else 0
            close = float(items['Close']) if items['Close'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(Date,Open,High,Low,'Adj Close','Volume','Close')
                    values (?,?, ?, ?,?,?,?);
            '''
            cursor.execute(sql,(date, open, high, low,adj_close,volume,close))

TypeError: tuple indices must be integers or slices, not str

In [49]:
print(data.columns)

MultiIndex([('Adj Close', '2330.TW'),
            (    'Close', '2330.TW'),
            (     'High', '2330.TW'),
            (      'Low', '2330.TW'),
            (     'Open', '2330.TW'),
            (   'Volume', '2330.TW')],
           names=['Price', 'Ticker'])


In [ ]:

conn = sqlite3.connect('record.db')
with conn:
    cursor = conn.cursor()

    for items in data.itertuples(index=False, name=None):  # 使用 name=None 簡化 tuple 結構
        date = items[0]  # 日期
        open_price = items[1]  # 開盤價
        high = items[2]  # 最高價
        low = items[3]  # 最低價
        adj_close = items[4]  # 調整收盤價
        volume = items[5]  # 成交量
        close = items[6]  # 收盤價

        sql = '''INSERT OR IGNORE INTO records(Date, Open, High, Low, "Adj Close", Volume, "Close")
                 VALUES (?, ?, ?, ?, ?, ?, ?);
              '''
        cursor.execute(sql, (date, open_price, high, low, adj_close, volume, close))


In [53]:
for items in data.itertuples(index=False, name=None):
    print(items)  # 檢查 tuple 的長度和內容


(437.9416198730469, 453.0, 453.5, 443.0, 446.0, 14885824)
(434.5579528808594, 449.5, 455.0, 448.5, 449.5, 19188422)
(443.2587890625, 458.5, 459.5, 455.0, 459.0, 23549581)
(443.2587890625, 458.5, 459.5, 455.0, 455.0, 20886011)
(465.0108337402344, 481.0, 481.0, 467.5, 468.0, 46666263)
(469.8446350097656, 486.0, 487.0, 483.0, 486.0, 32646014)
(468.3944396972656, 484.5, 488.0, 482.0, 487.0, 21051115)
(470.3279724121094, 486.5, 488.0, 484.0, 487.5, 20963689)
(483.3792419433594, 500.0, 509.0, 499.0, 507.0, 77606949)
(488.2130126953125, 505.0, 508.0, 503.0, 506.0, 34373918)
(486.27947998046875, 503.0, 504.0, 499.5, 504.0, 27770329)
(524.9498901367188, 543.0, 543.0, 534.0, 542.0, 124428936)
(504.6479187011719, 522.0, 538.0, 521.0, 537.0, 72530854)
(512.3820190429688, 530.0, 533.0, 522.0, 532.0, 33710990)
(522.0496215820312, 540.0, 542.0, 537.0, 538.0, 43077053)
(523.983154296875, 542.0, 542.0, 536.0, 540.0, 31635865)
(508.51483154296875, 526.0, 533.0, 526.0, 532.0, 30870136)
(505.6146850585937